# PLAN

 - Диалоговый датасет
     - BANKING77
     - CLINC150
     - HWU64
     - https://github.com/google-research-datasets/dstc8-schema-guided-dialogue
 - Генератор (XLM-R)
     - придумать как генерировать токен на другом языке
     - применить к задачке
 - LABSE для полученных и исходных предложений